In [1]:
import pandas as pd
import pathlib as pl
import duckdb as ddb

In [2]:
## Installing excel extension and loading that same extension in this command
## Allow for data to be exported to Excel (.xlsx) 
ddb.sql("INSTALL excel; LOAD excel;")

In [3]:
## File path and file name to original Closed Sales dataset
# filepath_closed_sales = "T:\Demand Planning\Carry ATS review\Exports\Closed Sales\Closed sales - size level.xlsx"
# filepath_closed_sales: pl.Path = pl.Path("T:", "Demand Planning", "Carry ATS review", "Exports", "Closed Sales")
# closed_sales_file_name: dict = {"ClosedSales": "Closed sales - size level.xlsx"}

## File path and file name to copy of Closed Sales dataset that has been formatted
## File has top row and 1st column row removed
# filepath_closed_sales = "C:\\Users\luan.nguyen\Documents\monolith\planning_accuracy_report\__raw_data\Closed sales - size level.xlsx"
filepath_closed_sales: pl.Path = pl.Path("C:\\", "Users", "luan.nguyen", "Documents", "monolith", "planning_accuracy_report", "__raw_data", "Closed sales - size level.xlsx")


In [4]:
## Load only the sheet into memory once
closed_sales_data = ddb.sql(f"""
    CREATE TEMPORARY TABLE closed_sales AS 
    SELECT * 
    FROM read_xlsx('{filepath_closed_sales}', header = TRUE, sheet = 'Basics Sales Data')
    UNION ALL
    SELECT * 
    FROM read_xlsx('{filepath_closed_sales}', header = TRUE, sheet = 'Basics Sales Data(1)')
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
## Save closed_sales parquet data
filepath_closed_sales_data_parquet: pl.Path = pl.Path("C:\\", "Users", "luan.nguyen", "Documents", "monolith", "planning_accuracy_report", "__raw_data")
filename_closed_sales_parquet: str = "closed_sales.parquet"
full_path_closed_sales_parquet: pl.Path = filepath_closed_sales_data_parquet.joinpath(filename_closed_sales_parquet)

In [6]:
## Because we're query this data frequently, making it into parquet
ddb.sql(f"""
    COPY closed_sales TO '{full_path_closed_sales_parquet}' (FORMAT PARQUET)
""")

In [7]:
## Get total count of rows to make sure data is intact
## As of 2025-08-24
count = ddb.sql(f"""
    SELECT COUNT(*)
    FROM '{full_path_closed_sales_parquet}'
""").fetchone()[0]

print(count)

1557186


In [10]:
closed_sales_data_1 = ddb.sql(f"""
    SELECT *
    FROM '{full_path_closed_sales_parquet}'
    WHERE "Date Start" >= '2024-10' AND "Date Start" <= '2024-12'
""")

In [11]:
count_2024 = ddb.sql(f"""
    SELECT COUNT(*)
    FROM 'closed_sales_data_1'
""").fetchone()[0]

print(count_2024)

0


In [12]:
closed_sales_data_2 = ddb.sql(f"""
    SELECT *
    FROM '{full_path_closed_sales_parquet}'
    WHERE "Date Start" >= '2025-01' AND "Date Start" <= '2025-10'
""")

In [13]:
count_2025 = ddb.sql(f"""
    SELECT COUNT(*)
    FROM 'closed_sales_data_2'
""").fetchone()[0]

print(count_2025)

1557186


In [21]:
with pd.ExcelWriter("__raw_data/closed_sales.xlsx") as writer:
    closed_sales_data_1.df().to_excel(writer, sheet_name="202410-202412", index = False)
    closed_sales_data_2.df().to_excel(writer, sheet_name="202501-202509", index = False)